In [1]:
import pickle
from collections import defaultdict
import pandas as pd
import sqlite3
import os
import numpy as np
import csv

In [2]:
#connect to database
db = sqlite3.connect('../raw_data/spotify.db')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS artist_info")

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [3]:
# #prep new table
# sqlite3.register_adapter(np.int64, lambda val: int(val))
# sqlite3.register_adapter(np.int32, lambda val: int(val))
#create a new table for artist information
cursor.execute('''CREATE TABLE IF NOT EXISTS artist_info
               (Artist_uri TEXT, genres TEXT, popularity INTEGER, artist_followers INTEGER)''')

In [5]:
directory = '../raw_data/artists/' #get directory of artist data
extract = defaultdict(list)   
for file in os.listdir(directory): #for each txt file in folder
    openFile = open('../raw_data/artists/'+file, 'rb') #open txt file
    loadFile = pickle.load(openFile) #load pickle object
    for artist in loadFile:
        uri = artist['uri'] #get artist uri
        genres = artist['genres'] #get genres
        pop = artist['popularity'] #get popularity (on a scale of 1-100)
        fan = artist['followers']['total'] #get total number of followers  
        
        #append to dictionary
        extract['Artist_uri'].append(uri)
        if len(genres) == 0:
            extract['genres'].append(NULL)
        else:
            extract['genres'].append(genres)
        extract['popularity'].append(pop)
        extract['artist_followers'].append(fan)

In [6]:
df_artist = pd.DataFrame.from_dict(extract)
df_artist.to_csv('../raw_data/artist_info.csv')

In [7]:
#write information from artist_info.csv into database
with open('../raw_data/artist_info.csv') as file:
    info = csv.reader(file, delimiter = ',')
    next(info, None)
    for row in info:
        index, uri, genres, pop, fan = row
        insert_into_db = (uri, genres, pop, fan)
        cursor.execute('''INSERT INTO artist_info
                         (Artist_uri, genres, popularity, artist_followers) 
                          VALUES (?,?,?,?)''', insert_into_db)

In [8]:
artist_cols = [col[1] for col in cursor.execute("PRAGMA table_info(artist_info)")]

In [12]:
query = '''SELECT * FROM artist_info WHERE genres LIKE 'NULL' LIMIT 10'''
viz_tables(artist_cols, query)

,Artist_uri,genres,popularity,artist_followers
0,spotify:artist:6A6v2KBuvVYBygLdyxDbXV,NULL,5,242
1,spotify:artist:397MkfYKFROVhksmvcf2nP,NULL,37,1218
2,spotify:artist:2ophM2hp2Y293Py2fhIrMw,NULL,2,184
3,spotify:artist:2ky8DpKY4vckr9Y2lci9tL,NULL,2,23
4,spotify:artist:1eyXYj1ubGcQYMFakHiPOM,NULL,2,12
5,spotify:artist:5MF6vg57YBVZvn8Mjrjes4,NULL,4,49
6,spotify:artist:2Fxjyl5l05sZ3tZggUzteZ,NULL,2,63
7,spotify:artist:0KPLDt4FvOoQ9ce9uph93k,NULL,5,87
8,spotify:artist:6A5tzCjyCuxulB7FDhSo8A,NULL,0,142
9,spotify:artist:3zmPeG94Y72IBB4XVqb9dM,NULL,0,19


In [13]:
db.commit()
db.close()